In [1]:
import glob
import numpy as np
import pandas as pd
import warnings

from dirichlet import mle
from scipy.stats import dirichlet


warnings.filterwarnings("ignore")
np.set_printoptions(suppress=True)

In [2]:
for filename in glob.glob("data\\*train*"):
    df = pd.read_csv(filename)
    compositions = df.iloc[:, df.columns.str.contains("z")]
    alpha = mle(compositions)

    print(dirichlet.rvs(alpha, size=10) * 100)
    break

[[ 0.08671844  0.23809099 52.01935702  3.42219386  3.57028088  0.44934314
   1.64900248  0.42826897  0.43192404  1.06586001  1.14628899  1.018177
   1.55440544  1.87657088  1.87875758  0.82065279  2.07903546  1.29617561
   2.03245471  0.46617954  0.99566333  0.23359654  0.192483   21.04851931]
 [ 0.51650635  0.80984352 54.27730342  4.8273065   0.8490621   1.16188042
   0.58561875  0.58209481  0.08860448  0.74093777  0.7259539   1.69842635
   4.20413947  1.39271135  1.8980042   1.73550991  1.71480806  1.0617523
   2.2076168   1.12142898  1.93457984  0.53297123  0.81227243 14.52066701]
 [ 0.01990515  0.74125757 47.34028699  3.03120082  2.083849    0.58032201
   0.87752726  0.07491376  0.62266821  0.58592567  0.68349896  3.04010597
   2.52704034  1.48926762  1.62043451  1.39969692  1.8582445   0.83132746
   2.06292326  2.04303055  0.94799781  2.4985852   1.2324636  21.80752685]
 [ 0.26416828  0.56704735 49.50794473  4.20790177  2.18506902  0.69238006
   3.32970099  1.05850581  0.22048431 